In [2]:
# region Import
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import statistics

sns.set()
pd.set_option('display.expand_frame_repr', False)

import warnings
warnings.filterwarnings('ignore')
# endregion

In [3]:
filePickle = 'C:/Users/phan/OneDrive - adesso Group/bofrost-data/F_umsatz_kopf.pickle'
with open(file=filePickle, mode='rb') as f:
   order: pd.DataFrame = pickle.load(f)

In [205]:
order['kunden_oid'].nunique()

3587423

In [81]:
kunde415693980 = order[order['kunden_oid'] == 415693980]
kunde415693980.head()

,kunden_oid,liefer_id,liefer_dtm,auftragswert_btr_eur
4,415693980,1075|5177632|VSS,2016-01-04,0.000000
403358,415693980,1075|5181844|VSS,2016-01-07,40.349998
2638275,415693980,1075|5194487|VSS,2016-01-27,0.000000
3924946,415693980,1075|5201859|VSS,2016-02-08,39.799999
5708773,415693980,1075|5211954|VSS,2016-02-24,0.000000


In [35]:
def predict(row: pd.DataFrame) -> float:
    if row['dateAllDifference'] >= row['min'] and row['dateAllDifference'] <= row['max']:
        return 1
    else:
        return 0
    
def truth(row: pd.DataFrame) -> float:
    if row['auftragswert_btr_eur'] > 0:
        return 1
    else:
        return 0

# kunde415693980

In [93]:
kunde415693980 = order[order['kunden_oid'] == 415693980]
kunde415693980 = kunde415693980.drop_duplicates(subset='liefer_dtm')
kunde415693980['auftragswert_btr_eur'] = kunde415693980['auftragswert_btr_eur'].round(0)
kunde415693980NotZero = kunde415693980[kunde415693980['auftragswert_btr_eur'] > 0]
kunde415693980['dateDifference'] = kunde415693980NotZero['liefer_dtm'].diff().dt.days
notNullIndices = list(kunde415693980['dateDifference'].dropna().index)

window = 3
dfLength = len(kunde415693980['dateDifference'].dropna())
for i in range(dfLength - window - 1):
    mean = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].mean().round(2)
    std = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].std().round(2)
    kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'meanWindow2'] = mean
    kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'stdWindow2'] = std
    
    nullIndices = kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1].index
    for nullIndex in nullIndices:
        kunde415693980.loc[nullIndex, 'dateAllDifference'] = (kunde415693980.loc[nullIndex, 'liefer_dtm'] - kunde415693980.loc[notNullIndices[i + window], 'liefer_dtm']).days



kunde415693980 = kunde415693980[['kunden_oid', 'liefer_id','auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'dateDifference', 'meanWindow2', 'stdWindow2']]
kunde415693980['min'] = kunde415693980['meanWindow2'] - kunde415693980['stdWindow2']
kunde415693980['max'] = kunde415693980['meanWindow2'] + kunde415693980['stdWindow2']
kunde415693980['predicted'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(predict, axis=1)
kunde415693980['truth'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(truth, axis=1)



print(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100)
print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateDifference', 'dateAllDifference','meanWindow2', 'min', 'max', 'predicted', 'truth']].dropna(), "\n")
print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateDifference', 'dateAllDifference','meanWindow2', 'min', 'max', 'predicted', 'truth']], "\n")
# print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']])

42.857142857142854
           auftragswert_btr_eur liefer_dtm  dateDifference  dateAllDifference  meanWindow2    min    max  predicted  truth
25269976                   35.0 2016-08-26            28.0               28.0        51.00  27.76  74.24          1      1
28266846                   38.0 2016-09-23            28.0               28.0        50.00  25.61  74.39          1      1
34175401                   36.0 2016-11-18            56.0               56.0        47.50  21.05  73.95          1      1
40065675                   66.0 2017-01-13            56.0               56.0        49.00  22.19  75.81          1      1
43175555                   73.0 2017-02-10            28.0               28.0        42.00  25.83  58.17          1      1
46021460                  104.0 2017-03-10            28.0               28.0        42.00  25.83  58.17          1      1
52734081                   60.0 2017-05-12            63.0               63.0        42.00  25.83  58.17          0     

In [105]:
kunde415693980 = order[order['kunden_oid'] == 60917561]
kunde415693980 = kunde415693980.drop_duplicates(subset='liefer_dtm')
kunde415693980['auftragswert_btr_eur'] = kunde415693980['auftragswert_btr_eur'].round(0)
kunde415693980NotZero = kunde415693980[kunde415693980['auftragswert_btr_eur'] > 0]
kunde415693980['dateDifference'] = kunde415693980NotZero['liefer_dtm'].diff().dt.days
notNullIndices = list(kunde415693980['dateDifference'].dropna().index)

window = 30
dfLength = len(kunde415693980['dateDifference'].dropna())
for i in range(dfLength - window - 1):
    mean = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].mean().round(2)
    std = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].std().round(2)
    kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'meanWindow2'] = mean
    kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'stdWindow2'] = std
    
    nullIndices = kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1].index
    for nullIndex in nullIndices:
        kunde415693980.loc[nullIndex, 'dateAllDifference'] = (kunde415693980.loc[nullIndex, 'liefer_dtm'] - kunde415693980.loc[notNullIndices[i + window], 'liefer_dtm']).days



kunde415693980 = kunde415693980[['kunden_oid', 'liefer_id','auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'dateDifference', 'meanWindow2', 'stdWindow2']]
kunde415693980['min'] = kunde415693980['meanWindow2'] - kunde415693980['stdWindow2']
kunde415693980['max'] = kunde415693980['meanWindow2'] + kunde415693980['stdWindow2']
kunde415693980['predicted'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(predict, axis=1)
kunde415693980['truth'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(truth, axis=1)



print(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100)
print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateDifference', 'dateAllDifference','meanWindow2', 'min', 'max', 'predicted', 'truth']].dropna().head(60), "\n")
print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateDifference', 'dateAllDifference','meanWindow2', 'min', 'max', 'predicted', 'truth']].tail(60), "\n")
# print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']])

80.0
          auftragswert_btr_eur liefer_dtm  dateDifference  dateAllDifference  meanWindow2   min    max  predicted  truth
16365349                  28.0 2016-06-04             2.0                2.0         4.71  1.21   8.21          1      1
16768522                   6.0 2016-06-08             4.0                4.0         4.39  1.13   7.65          1      1
19526197                  10.0 2016-07-04            26.0               26.0         4.45  1.22   7.68          0      1
19653407                   7.0 2016-07-05             1.0                1.0         5.19  0.16  10.22          1      1
19806665                  54.0 2016-07-06             1.0                1.0         5.19  0.16  10.22          1      1
20880903                   9.0 2016-07-15             9.0                9.0         5.19  0.16  10.22          1      1
21010523                  17.0 2016-07-18             3.0                3.0         5.45  0.44  10.46          1      1
21174396                  2

# Test for all Kunden

In [33]:
with open(file='kundenIndices.pickle', mode='rb') as f:
    kundenOids = pickle.load(f)

In [42]:
for kundenOid in kundenOids:
    kunde415693980 = order[order['kunden_oid'] == kundenOid]
    kunde415693980 = kunde415693980.drop_duplicates(subset='liefer_dtm')
    kunde415693980['auftragswert_btr_eur'] = kunde415693980['auftragswert_btr_eur'].round(0)
    kunde415693980NotZero = kunde415693980[kunde415693980['auftragswert_btr_eur'] > 0]
    kunde415693980['dateDifference'] = kunde415693980NotZero['liefer_dtm'].diff().dt.days
    notNullIndices = list(kunde415693980['dateDifference'].dropna().index)

    window = 30
    dfLength = len(kunde415693980['dateDifference'].dropna())
    for i in range(dfLength - window-1):
        mean = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].mean().round(2)
        std = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].std().round(2)
        kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'meanWindow2'] = mean
        kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'stdWindow2'] = std

        nullIndices = kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1].index
        for nullIndex in nullIndices:
            kunde415693980.loc[nullIndex, 'dateAllDifference'] = (kunde415693980.loc[nullIndex, 'liefer_dtm'] - kunde415693980.loc[notNullIndices[i + window], 'liefer_dtm']).days


    kunde415693980 = kunde415693980[['kunden_oid', 'liefer_id','auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'dateDifference', 'meanWindow2', 'stdWindow2']]
    kunde415693980['min'] = kunde415693980['meanWindow2'] - kunde415693980['stdWindow2']
    kunde415693980['max'] = kunde415693980['meanWindow2'] + kunde415693980['stdWindow2']
    kunde415693980['predicted'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(predict, axis=1)
    kunde415693980['truth'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(truth, axis=1)

    print(round(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100, 2))
#     print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna(), "\n")
    # print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']])

58.44
55.94
64.29
62.75
70.59
66.34
82.64
58.33
66.67
84.83
75.21
54.17
40.0
52.17
78.45
58.33
68.42
56.58
84.46
74.76
66.67
75.93
72.09
60.98
53.12
59.09
46.43
67.24
56.36
75.7
73.21
39.74
70.71
67.14
63.64
60.58
71.23
74.19
47.44
66.12
60.29
82.61
68.85
61.39
68.82
70.83
50.62
70.15
73.33
70.59
59.49
61.54
67.47
50.0
72.07
71.72
56.25


In [67]:
kunde415693980 = order[order['kunden_oid'] == 415693980]
kunde415693980 = kunde415693980.drop_duplicates(subset='liefer_dtm')
kunde415693980['auftragswert_btr_eur'] = kunde415693980['auftragswert_btr_eur'].round(0)
kunde415693980NotZero = kunde415693980[kunde415693980['auftragswert_btr_eur'] > 0]
kunde415693980['dateDifference'] = kunde415693980NotZero['liefer_dtm'].diff().dt.days
notNullIndices = list(kunde415693980['dateDifference'].dropna().index)


# window = 20
dfLength = len(kunde415693980['dateDifference'].dropna())
for window in range(dfLength - 2, 1, -1):
    for i in range(dfLength - window-1):
        mean = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].mean().round(2)
        std = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].std().round(2)
        kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'meanWindow2'] = mean
        kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'stdWindow2'] = std

        nullIndices = kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1].index
        for nullIndex in nullIndices:
            kunde415693980.loc[nullIndex, 'dateAllDifference'] = (kunde415693980.loc[nullIndex, 'liefer_dtm'] - kunde415693980.loc[notNullIndices[i + window], 'liefer_dtm']).days

    kunde415693980 = kunde415693980[['kunden_oid', 'liefer_id','auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'dateDifference', 'meanWindow2', 'stdWindow2']]
    kunde415693980['min'] = kunde415693980['meanWindow2'] - kunde415693980['stdWindow2']
    kunde415693980['max'] = kunde415693980['meanWindow2'] + kunde415693980['stdWindow2']
    kunde415693980['predicted'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(predict, axis=1)
    kunde415693980['truth'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(truth, axis=1)

    # kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]

    print(f'window = {window} -> ', round(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100, 2), "%")
#     print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna(), "\n")
#     print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']])

window = 27 ->  50.0 %
window = 26 ->  66.67 %
window = 25 ->  50.0 %
window = 24 ->  60.0 %
window = 23 ->  66.67 %
window = 22 ->  71.43 %
window = 21 ->  66.67 %
window = 20 ->  70.0 %
window = 19 ->  66.67 %
window = 18 ->  69.23 %
window = 17 ->  64.29 %
window = 16 ->  62.5 %
window = 15 ->  64.71 %
window = 14 ->  57.89 %
window = 13 ->  65.0 %
window = 12 ->  54.55 %
window = 11 ->  60.87 %
window = 10 ->  50.0 %
window = 9 ->  44.44 %
window = 8 ->  50.0 %
window = 7 ->  48.28 %
window = 6 ->  51.61 %
window = 5 ->  48.48 %
window = 4 ->  50.0 %
window = 3 ->  42.86 %
window = 2 ->  43.59 %


In [135]:
testResultsDf = pd.DataFrame(columns=['kundenOid', 'window', 'acc', 'testSamples'])

In [137]:
# testResults = pd.DataFrame(columns=['kunden_oid', 'window_size', 'acc'])
# print(testResult)
for kundenOid in kundenOids:
    kunde415693980 = order[order['kunden_oid'] == kundenOid]
    kunde415693980 = kunde415693980.drop_duplicates(subset='liefer_dtm')
    kunde415693980['auftragswert_btr_eur'] = kunde415693980['auftragswert_btr_eur'].round(0)
    kunde415693980NotZero = kunde415693980[kunde415693980['auftragswert_btr_eur'] > 0]
    kunde415693980['dateDifference'] = kunde415693980NotZero['liefer_dtm'].diff().dt.days
    notNullIndices = list(kunde415693980['dateDifference'].dropna().index)

    
    
    dfLength = len(kunde415693980['dateDifference'].dropna())
    for window in range(int(dfLength*0.7), 5, -1):
        for i in range(dfLength - window-1):
            mean = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].mean().round(2)
            std = kunde415693980["dateDifference"].dropna().loc[notNullIndices[i]:notNullIndices[i + window]].std().round(2)
            kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'meanWindow2'] = mean
            kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1, 'stdWindow2'] = std

            nullIndices = kunde415693980.loc[notNullIndices[i + window] + 1:notNullIndices[i + window + 1] + 1].index
            for nullIndex in nullIndices:
                kunde415693980.loc[nullIndex, 'dateAllDifference'] = (kunde415693980.loc[nullIndex, 'liefer_dtm'] - kunde415693980.loc[notNullIndices[i + window], 'liefer_dtm']).days


        kunde415693980 = kunde415693980[['kunden_oid', 'liefer_id','auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'dateDifference', 'meanWindow2', 'stdWindow2']]
        kunde415693980['min'] = kunde415693980['meanWindow2'] - kunde415693980['stdWindow2']
        kunde415693980['max'] = kunde415693980['meanWindow2'] + kunde415693980['stdWindow2']
        kunde415693980['predicted'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(predict, axis=1)
        kunde415693980['truth'] = kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max']].apply(truth, axis=1)

        testSamples = len(kunde415693980.dropna())
        acc = round(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100, 2)
        singleTestResult = pd.DataFrame({
            'kundenOid': [kundenOid], 
            'window': [window], 
            'acc': [acc],
            'testSamples': [testSamples]})
        testResultsDf = pd.concat(objs=[testResultsDf, singleTestResult], axis=0)
    print(testResultsDf[testResultsDf['kundenOid'] == kundenOid].head(60), '\n')
        #         pd.concat(objs=, axis=)
#         print(f'kunde{kundenOid} window = {window} -> ', f'testSamples = {len(kunde415693980.dropna())}  ', round(len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()[kunde415693980['predicted'] == kunde415693980['truth']]) / len(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna()) * 100, 2), '%')
    #     print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].dropna(), "\n")
#         print(kunde415693980[['auftragswert_btr_eur', 'liefer_dtm', 'dateAllDifference', 'min', 'max', 'predicted', 'truth']].head(60))

  kundenOid window    acc testSamples
0     24005     58  62.07          25
0     24005     57  60.00          26
0     24005     56  61.29          27
0     24005     55  60.61          28
0     24005     54  58.82          29
0     24005     53  57.14          30
0     24005     52  56.76          31
0     24005     51  55.26          32
0     24005     50  56.41          33
0     24005     49  57.50          34
0     24005     48  61.90          35
0     24005     47  60.47          36
0     24005     46  61.36          37
0     24005     45  62.22          38
0     24005     44  63.04          39
0     24005     43  61.70          40
0     24005     42  60.42          41
0     24005     41  60.00          42
0     24005     40  63.46          43
0     24005     39  66.04          44
0     24005     38  64.81          45
0     24005     37  67.27          46
0     24005     36  64.29          47
0     24005     35  66.67          48
0     24005     34  65.52          49
0     24005 

  kundenOid window    acc testSamples
0  30250352     98  82.93          41
0  30250352     97  83.33          42
0  30250352     96  81.82          43
0  30250352     95  82.61          44
0  30250352     94  82.98          45
0  30250352     93  83.33          46
0  30250352     92  83.67          47
0  30250352     91  84.00          48
0  30250352     90  84.31          49
0  30250352     89  82.69          50
0  30250352     88  84.91          51
0  30250352     87  87.04          52
0  30250352     86  87.27          53
0  30250352     85  85.71          54
0  30250352     84  87.72          55
0  30250352     83  86.21          56
0  30250352     82  86.44          57
0  30250352     81  85.00          58
0  30250352     80  85.25          59
0  30250352     79  83.87          60
0  30250352     78  80.95          61
0  30250352     77  85.94          62
0  30250352     76  84.62          63
0  30250352     75  83.33          64
0  30250352     74  82.09          65
0  30250352 

  kundenOid window    acc testSamples
0  41082073     95  78.57          41
0  41082073     94  76.74          42
0  41082073     93  75.00          43
0  41082073     92  75.56          44
0  41082073     91  76.09          45
0  41082073     90  76.60          46
0  41082073     89  77.08          47
0  41082073     88  75.51          48
0  41082073     87  76.00          49
0  41082073     86  74.51          50
0  41082073     85  75.00          51
0  41082073     84  75.47          52
0  41082073     83  75.93          53
0  41082073     82  76.36          54
0  41082073     81  76.79          55
0  41082073     80  77.19          56
0  41082073     79  77.59          57
0  41082073     78  77.97          58
0  41082073     77  78.33          59
0  41082073     76  78.69          60
0  41082073     75  79.03          61
0  41082073     74  79.37          62
0  41082073     73  79.69          63
0  41082073     72  80.00          64
0  41082073     71  80.30          65
0  41082073 

  kundenOid window    acc testSamples
0  55923042     61  63.64          26
0  55923042     60  62.86          27
0  55923042     59  63.89          28
0  55923042     58  64.86          29
0  55923042     57  63.16          30
0  55923042     56  64.10          31
0  55923042     55  62.50          32
0  55923042     54  63.41          33
0  55923042     53  64.29          34
0  55923042     52  61.36          35
0  55923042     51  62.22          36
0  55923042     50  61.70          37
0  55923042     49  60.42          38
0  55923042     48  63.27          39
0  55923042     47  64.71          40
0  55923042     46  65.38          41
0  55923042     45  66.04          42
0  55923042     44  66.67          43
0  55923042     43  67.27          44
0  55923042     42  69.64          45
0  55923042     41  65.52          46
0  55923042     40  66.10          47
0  55923042     39  66.67          48
0  55923042     38  65.57          49
0  55923042     37  67.74          50
0  55923042 

   kundenOid window    acc testSamples
0  111359400     84  72.97          36
0  111359400     83  73.68          37
0  111359400     82  74.36          38
0  111359400     81  72.50          39
0  111359400     80  75.61          40
0  111359400     79  73.81          41
0  111359400     78  67.44          42
0  111359400     77  68.18          43
0  111359400     76  68.89          44
0  111359400     75  69.57          45
0  111359400     74  70.21          46
0  111359400     73  68.75          47
0  111359400     72  67.35          48
0  111359400     71  66.00          49
0  111359400     70  67.31          50
0  111359400     69  67.92          51
0  111359400     68  68.52          52
0  111359400     67  69.09          53
0  111359400     66  71.43          54
0  111359400     65  71.93          55
0  111359400     64  70.69          56
0  111359400     63  71.19          57
0  111359400     62  70.00          58
0  111359400     61  70.49          59
0  111359400     60  67.7

...

   kundenOid window    acc testSamples
0  426770249     63  64.29          27
0  426770249     62  62.07          28
0  426770249     61  60.00          29
0  426770249     60  61.29          30
0  426770249     59  62.50          31
0  426770249     58  63.64          32
0  426770249     57  64.71          33
0  426770249     56  65.71          34
0  426770249     55  63.89          35
0  426770249     54  64.86          36
0  426770249     53  68.42          37
0  426770249     52  69.23          38
0  426770249     51  70.00          39
0  426770249     50  70.73          40
0  426770249     49  69.05          41
0  426770249     48  69.77          42
0  426770249     47  70.45          43
0  426770249     46  71.11          44
0  426770249     45  67.39          45
0  426770249     44  68.09          46
0  426770249     43  66.67          47
0  426770249     42  67.35          48
0  426770249     41  68.00          49
0  426770249     40  66.67          50
0  426770249     39  63.4

   kundenOid window    acc testSamples
0  460664257     91  73.17          39
0  460664257     90  73.81          40
0  460664257     89  74.42          41
0  460664257     88  75.00          42
0  460664257     87  75.56          43
0  460664257     86  76.09          44
0  460664257     85  78.72          45
0  460664257     84  77.08          46
0  460664257     83  77.55          47
0  460664257     82  78.00          48
0  460664257     81  78.43          49
0  460664257     80  78.85          50
0  460664257     79  79.25          51
0  460664257     78  79.63          52
0  460664257     77  80.00          53
0  460664257     76  80.36          54
0  460664257     75  80.70          55
0  460664257     74  82.76          56
0  460664257     73  81.36          57
0  460664257     72  81.67          58
0  460664257     71  80.33          59
0  460664257     70  80.65          60
0  460664257     69  80.95          61
0  460664257     68  81.25          62
0  460664257     67  83.0

   kundenOid window    acc testSamples
0  515350433     80  61.76          34
0  515350433     79  65.71          35
0  515350433     78  63.89          36
0  515350433     77  64.86          37
0  515350433     76  68.42          38
0  515350433     75  69.23          39
0  515350433     74  67.50          40
0  515350433     73  70.73          41
0  515350433     72  69.05          42
0  515350433     71  69.77          43
0  515350433     70  70.45          44
0  515350433     69  68.89          45
0  515350433     68  71.74          46
0  515350433     67  72.34          47
0  515350433     66  72.92          48
0  515350433     65  71.43          49
0  515350433     64  70.00          50
0  515350433     63  66.67          51
0  515350433     62  65.38          52
0  515350433     61  66.04          53
0  515350433     60  68.52          54
0  515350433     59  67.86          55
0  515350433     58  68.42          56
0  515350433     57  67.24          57
0  515350433     56  66.1

   kundenOid window    acc testSamples
0  653992234     74  87.10          31
0  653992234     73  87.50          32
0  653992234     72  90.91          33
0  653992234     71  88.24          34
0  653992234     70  88.57          35
0  653992234     69  88.89          36
0  653992234     68  89.19          37
0  653992234     67  86.84          38
0  653992234     66  87.18          39
0  653992234     65  87.50          40
0  653992234     64  87.80          41
0  653992234     63  85.71          42
0  653992234     62  86.05          43
0  653992234     61  86.36          44
0  653992234     60  86.67          45
0  653992234     59  86.96          46
0  653992234     58  87.23          47
0  653992234     57  87.50          48
0  653992234     56  83.67          49
0  653992234     55  84.00          50
0  653992234     54  84.31          51
0  653992234     53  84.62          52
0  653992234     52  83.02          53
0  653992234     51  83.33          54
0  653992234     50  81.8

   kundenOid window    acc testSamples
0  740182010     89  82.50          38
0  740182010     88  82.93          39
0  740182010     87  83.33          40
0  740182010     86  81.40          41
0  740182010     85  81.82          42
0  740182010     84  82.22          43
0  740182010     83  82.61          44
0  740182010     82  85.11          45
0  740182010     81  85.42          46
0  740182010     80  85.71          47
0  740182010     79  86.00          48
0  740182010     78  86.27          49
0  740182010     77  86.54          50
0  740182010     76  84.91          51
0  740182010     75  85.19          52
0  740182010     74  83.64          53
0  740182010     73  83.93          54
0  740182010     72  85.96          55
0  740182010     71  86.21          56
0  740182010     70  86.44          57
0  740182010     69  85.00          58
0  740182010     68  85.25          59
0  740182010     67  85.48          60
0  740182010     66  84.13          61
0  740182010     65  84.3

In [145]:
testResultsDf = testResultsDf.reset_index()[['kundenOid', 'window', 'acc', 'testSamples']]

In [164]:
accByKundeOid = testResultsDf.groupby(by='kundenOid').agg(
    maxAcc= pd.NamedAgg(column='acc', aggfunc=np.max)
)
accByKundeOid.head()

,maxAcc
kundenOid,
24005,67.27
10327048,62.42
12788787,70.37
16926734,70.27
28929469,85.71


In [178]:
testResultsDf.nlargest(n=1, columns='acc')

,kundenOid,window,acc,testSamples
1026,110326551,80,93.18,88


In [185]:
testResultsDf['kundenOid'].nunique()

57

In [181]:
testResultsDf.groupby(by='kundenOid')['acc'].transform(lambda acc: np.max(acc))

0       67.27
1       67.27
2       67.27
3       67.27
4       67.27
        ...  
3241    60.24
3242    60.24
3243    60.24
3244    60.24
3245    60.24
Name: acc, Length: 3246, dtype: float64

In [183]:
testResultsDf.groupby(by='kundenOid').agg(
    window=pd.NamedAgg(column='window')
)

TypeError: __new__() missing 1 required positional argument: 'aggfunc'

In [177]:
testResultsDf.head()

,kundenOid,window,acc,testSamples
0,24005,58,62.07,25
1,24005,57,60.00,26
2,24005,56,61.29,27
3,24005,55,60.61,28
4,24005,54,58.82,29


In [202]:
testResultsDf.sort_values(by='acc', ascending=False).drop_duplicates(subset='kundenOid').sort_values(by='kundenOid')

,kundenOid,window,acc,testSamples
21,24005,37,67.27,46
127,10327048,8,62.42,109
146,12788787,7,70.37,26
155,16926734,28,70.27,21
181,28929469,23,85.71,14
199,29967356,84,86.11,36
292,30250352,84,87.72,55
371,30572512,47,69.57,20
428,35975686,47,69.09,42
475,39973402,109,92.59,54


In [190]:
testResultsDf.groupby(by='kundenOid').agg(
    acc=pd.NamedAgg(column='acc', aggfunc=np.max)
)

,acc
kundenOid,
24005,67.27
10327048,62.42
12788787,70.37
16926734,70.27
28929469,85.71
29967356,86.11
30250352,87.72
30572512,69.57
35975686,69.09


In [171]:
with open(file='testResultsDf', mode='wb') as f:
    pickle.dump(testResult, f)

In [173]:
accByKundeOid.mean()

maxAcc    72.615439
dtype: float64

In [184]:
accByKundeOid.std()

maxAcc    10.840908
dtype: float64